## Hypotheses: 
- age has an ambigugos effect on preparedness
- income has an ambiguous effect on preparedness
- education has a positive effect on preparedness
- homeownership has a positive effect on preparedness

In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df = pd.read_csv(path)
df.head()

,id,state,zipcode,geographic_division,census_region,county,experience,supplies,insured,involved,...,sex,education,race,homeownership,income,rentmortgage,rurality,hazard_weight,geometry,zip_count
0,1,Washington,98033,Pacific,West,King,0.0,0,0,1,...,0.0,4,White,1,87500,1250.0,1,0.255097,POLYGON ((-13608075.601186747 6055357.24645823...,1
1,2,Florida,33950,South Atlantic,South,Charlotte,1.0,0,1,1,...,1.0,5,White,1,175000,3000.0,1,2.836145,POLYGON ((-9139287.781401768 3112400.961508024...,1
2,3,New Jersey,7031,Middle Atlantic,Northeast,Bergen,1.0,0,0,1,...,1.0,1,White,0,47500,1250.0,1,1.165971,POLYGON ((-8254065.171859486 4980899.960122439...,1
3,5,Pennsylvania,19148,Middle Atlantic,Northeast,Philadelphia,1.0,0,1,0,...,0.0,4,White,1,125000,1250.0,1,0.624890,POLYGON ((-8368433.592186097 4851218.465580212...,1
4,7,Florida,32976,South Atlantic,South,Brevard,0.0,1,1,0,...,0.0,2,White,1,20000,250.0,1,0.695075,POLYGON ((-8971557.027329512 3234931.888104651...,1


#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication


In [3]:
# define the regression model 
def model (function, data):
    model = smf.logit(function, data=data).fit()
    print(model.summary()) # print model summary
    print('BIC:', model.bic) # print BIC additionally to defaults model fit parameters
    # print odds ratios for output = 0 and output = 1
    #print('likelihood of adaptation if household is not insured = ', 1 / (1 + np.exp(-model.params['Intercept'])))
    #print('likelihood of adaptation if household is insured = ', 1 / (1 + np.exp(-(model.params['Intercept']+model.params['insured']))))

    # compute odds ratio
    params = model.params
    conf = model.conf_int()
    odds_ratios = pd.DataFrame({
        "OR": np.exp(params),
        "Lower CI": np.exp(conf[0]),
        "Upper CI": np.exp(conf[1])
        })
    odds_ratios.index.name = 'Variable'
    odds_ratios.reset_index(inplace=True)
    # compute marginal effects
    marginal_effects = model.get_margeff().summary()

    return odds_ratios, marginal_effects 
    

In [4]:
def probit (function, data):
    model = smf.probit(function, data=data).fit()
    print(model.summary())
    print('BIC=', model.bic)
    marginal = model.get_margeff().summary()
    return marginal

In [5]:
odds, marg_eff = model('made_safer ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.469262
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.01804
Time:                        17:59:47   Log-Likelihood:                -221.49
converged:                       True   LL-Null:                       -225.56
Covariance Type:            nonrobust   LLR p-value:                   0.08659
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.9408      0.377     -2.496      0.013      -1.680      -0.202
age              -0.

In [6]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.390308,0.186427,0.817161
1,age,0.992551,0.980484,1.004766
2,income,0.999995,0.999989,1.000001
3,education,0.941131,0.786034,1.126831
4,homeownership,1.770363,1.042064,3.007670


In [7]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:             made_safer
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0011      0.001     -1.201      0.230      -0.003       0.001
income        -7.687e-07   4.38e-07     -1.754      0.079   -1.63e-06    9.02e-08
education        -0.0090      0.014     -0.661      0.509      -0.036       0.018
homeownership     0.0844      0.040      2.128      0.033       0.007       0.162
=================================================================================
"""

In [8]:
odds, marg_eff = model('insured ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.504342
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                insured   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02859
Time:                        17:59:47   Log-Likelihood:                -238.05
converged:                       True   LL-Null:                       -245.06
Covariance Type:            nonrobust   LLR p-value:                  0.007252
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.1160      0.368     -3.034      0.002      -1.837      -0.395
age              -0.

In [9]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.327588,0.159316,0.673590
1,age,0.984893,0.973392,0.996531
2,income,0.999998,0.999993,1.000003
3,education,1.114872,0.940759,1.321209
4,homeownership,2.069913,1.243174,3.446453


In [10]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                insured
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0025      0.001     -2.581      0.010      -0.004      -0.001
income        -3.193e-07   4.12e-07     -0.776      0.438   -1.13e-06    4.88e-07
education         0.0177      0.014      1.260      0.208      -0.010       0.045
homeownership     0.1187      0.042      2.850      0.004       0.037       0.200
=================================================================================
"""

In [11]:
odds, marg_eff = model('learned_routes ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.534859
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:         learned_routes   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:               0.0007334
Time:                        17:59:47   Log-Likelihood:                -252.45
converged:                       True   LL-Null:                       -252.64
Covariance Type:            nonrobust   LLR p-value:                    0.9848
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.1765      0.350     -3.359      0.001      -1.863      -0.490
age               0.

In [12]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.308371,0.155229,0.612597
1,age,1.000332,0.989231,1.011558
2,income,1.000001,0.999996,1.000005
3,education,0.985099,0.837033,1.159357
4,homeownership,0.871851,0.540796,1.405566


In [13]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:         learned_routes
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age             5.82e-05      0.001      0.058      0.953      -0.002       0.002
income           1.2e-07   4.19e-07      0.286      0.775   -7.01e-07    9.41e-07
education        -0.0026      0.015     -0.181      0.857      -0.031       0.026
homeownership    -0.0240      0.043     -0.563      0.573      -0.108       0.060
=================================================================================
"""

In [14]:
odds, marg_eff = model('supplies ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.513828
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               supplies   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02562
Time:                        17:59:47   Log-Likelihood:                -242.53
converged:                       True   LL-Null:                       -248.90
Covariance Type:            nonrobust   LLR p-value:                   0.01255
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2106      0.350     -0.601      0.548      -0.897       0.476
age              -0.

In [15]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.810133,0.407746,1.609616
1,age,0.982039,0.970485,0.993730
2,income,0.999997,0.999992,1.000002
3,education,0.936461,0.791580,1.107860
4,homeownership,1.466959,0.894872,2.404779


In [16]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               supplies
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0030      0.001     -3.070      0.002      -0.005      -0.001
income        -4.789e-07   4.43e-07     -1.080      0.280   -1.35e-06     3.9e-07
education        -0.0110      0.014     -0.767      0.443      -0.039       0.017
homeownership     0.0641      0.042      1.528      0.126      -0.018       0.146
=================================================================================
"""

In [17]:
odds, marg_eff = model('involved ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.276026
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:               involved   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08046
Time:                        17:59:47   Log-Likelihood:                -130.28
converged:                       True   LL-Null:                       -141.68
Covariance Type:            nonrobust   LLR p-value:                 0.0001388
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.9044      0.512     -1.767      0.077      -1.908       0.099
age              -0.

In [18]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.404765,0.148451,1.103633
1,age,0.963939,0.944829,0.983436
2,income,1.000002,0.999995,1.000009
3,education,1.192269,0.936645,1.517658
4,homeownership,0.475929,0.227126,0.997279


In [19]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               involved
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0028      0.001     -3.457      0.001      -0.004      -0.001
income         1.772e-07   2.82e-07      0.628      0.530   -3.76e-07     7.3e-07
education         0.0135      0.010      1.423      0.155      -0.005       0.032
homeownership    -0.0572      0.029     -1.948      0.051      -0.115       0.000
=================================================================================
"""

In [20]:
odds, marg_eff = model('made_plan ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.519816
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              made_plan   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                0.009207
Time:                        17:59:47   Log-Likelihood:                -245.35
converged:                       True   LL-Null:                       -247.63
Covariance Type:            nonrobust   LLR p-value:                    0.3355
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.6242      0.349     -1.790      0.073      -1.307       0.059
age              -0.

In [21]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.535710,0.270513,1.060893
1,age,0.989513,0.978128,1.001030
2,income,1.000001,0.999996,1.000006
3,education,0.932271,0.789553,1.100787
4,homeownership,0.993420,0.610383,1.616826


In [22]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:              made_plan
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0018      0.001     -1.799      0.072      -0.004       0.000
income         1.733e-07   4.13e-07      0.420      0.675   -6.36e-07    9.83e-07
education        -0.0118      0.014     -0.829      0.407      -0.040       0.016
homeownership    -0.0011      0.042     -0.027      0.979      -0.083       0.081
=================================================================================
"""

In [23]:
odds, marg_eff = model('practiced_drills ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.263724
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       practiced_drills   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.05816
Time:                        17:59:47   Log-Likelihood:                -124.48
converged:                       True   LL-Null:                       -132.16
Covariance Type:            nonrobust   LLR p-value:                  0.003986
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.9130      0.527     -1.732      0.083      -1.946       0.120
age              -0.

In [24]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.401304,0.142820,1.127605
1,age,0.963585,0.944268,0.983296
2,income,0.999997,0.999989,1.000005
3,education,1.112412,0.862687,1.434426
4,homeownership,1.217601,0.576819,2.570220


In [25]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:       practiced_drills
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0027      0.001     -3.391      0.001      -0.004      -0.001
income        -1.972e-07      3e-07     -0.658      0.510   -7.84e-07     3.9e-07
education         0.0076      0.009      0.819      0.413      -0.011       0.026
homeownership     0.0141      0.027      0.516      0.606      -0.039       0.068
=================================================================================
"""

In [26]:
odds, marg_eff = model('alerts ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.482979
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 alerts   No. Observations:                  472
Model:                          Logit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                0.002371
Time:                        17:59:47   Log-Likelihood:                -227.97
converged:                       True   LL-Null:                       -228.51
Covariance Type:            nonrobust   LLR p-value:                    0.8969
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.7218      0.384     -4.483      0.000      -2.474      -0.969
age               0.

In [27]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.178750,0.084209,0.379432
1,age,1.003488,0.991651,1.015465
2,income,0.999999,0.999993,1.000004
3,education,1.034148,0.868006,1.232090
4,homeownership,1.155815,0.691381,1.932229


In [28]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 alerts
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age               0.0005      0.001      0.575      0.565      -0.001       0.002
income        -2.082e-07   4.01e-07     -0.519      0.604   -9.95e-07    5.78e-07
education         0.0051      0.014      0.376      0.707      -0.022       0.032
homeownership     0.0221      0.040      0.553      0.581      -0.056       0.101
=================================================================================
"""

In [29]:
odds, marg_eff = model('family_communication ~ age + income + education + homeownership', data=df)

Optimization terminated successfully.
         Current function value: 0.320466
         Iterations 7
                            Logit Regression Results                            
Dep. Variable:     family_communication   No. Observations:                  472
Model:                            Logit   Df Residuals:                      467
Method:                             MLE   Df Model:                            4
Date:                  Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02532
Time:                          17:59:47   Log-Likelihood:                -151.26
converged:                         True   LL-Null:                       -155.19
Covariance Type:              nonrobust   LLR p-value:                   0.09687
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.5309      0.484     -3.165      0.002      -2.479      -0.583
age 

In [30]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.216346,0.083834,0.558313
1,age,0.979207,0.963177,0.995504
2,income,0.999999,0.999992,1.000006
3,education,1.171151,0.932632,1.470670
4,homeownership,1.059362,0.543746,2.063921


In [31]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects        
=======================================
Dep. Variable:     family_communication
Method:                            dydx
At:                             overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0019      0.001     -2.454      0.014      -0.003      -0.000
income        -1.002e-07   3.12e-07     -0.321      0.748   -7.12e-07    5.12e-07
education         0.0142      0.010      1.354      0.176      -0.006       0.035
homeownership     0.0052      0.031      0.169      0.865      -0.055       0.065
=================================================================================
"""

## Let's repeat with probit

In [32]:
probit('made_safer ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.469319
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.01792
Time:                        18:01:02   Log-Likelihood:                -221.52
converged:                       True   LL-Null:                       -225.56
Covariance Type:            nonrobust   LLR p-value:                   0.08849
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.5957      0.212     -2.807      0.005      -1.012      -0.180
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:             made_safer
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0011      0.001     -1.171      0.241      -0.003       0.001
income        -7.547e-07   4.25e-07     -1.777      0.076   -1.59e-06    7.76e-08
education        -0.0089      0.014     -0.661      0.508      -0.035       0.018
homeownership     0.0839      0.039      2.128      0.033       0.007       0.161
=================================================================================
"""

In [33]:
probit('insured ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.504655
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                insured   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02799
Time:                        18:01:15   Log-Likelihood:                -238.20
converged:                       True   LL-Null:                       -245.06
Covariance Type:            nonrobust   LLR p-value:                  0.008252
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.6865      0.211     -3.249      0.001      -1.101      -0.272
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                insured
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0024      0.001     -2.524      0.012      -0.004      -0.001
income         -3.19e-07   4.12e-07     -0.775      0.438   -1.13e-06    4.88e-07
education         0.0174      0.014      1.232      0.218      -0.010       0.045
homeownership     0.1169      0.041      2.832      0.005       0.036       0.198
=================================================================================
"""

In [34]:
probit('learned_routes ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.534858
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:         learned_routes   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:               0.0007357
Time:                        18:01:25   Log-Likelihood:                -252.45
converged:                       True   LL-Null:                       -252.64
Covariance Type:            nonrobust   LLR p-value:                    0.9847
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.7198      0.204     -3.528      0.000      -1.120      -0.320
age               0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:         learned_routes
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age             5.33e-05      0.001      0.053      0.957      -0.002       0.002
income         1.225e-07   4.21e-07      0.291      0.771   -7.02e-07    9.47e-07
education        -0.0026      0.015     -0.177      0.859      -0.032       0.026
homeownership    -0.0241      0.043     -0.562      0.574      -0.108       0.060
=================================================================================
"""

In [35]:
probit('supplies ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.513814
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:               supplies   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02564
Time:                        18:01:29   Log-Likelihood:                -242.52
converged:                       True   LL-Null:                       -248.90
Covariance Type:            nonrobust   LLR p-value:                   0.01248
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.1608      0.205     -0.786      0.432      -0.562       0.240
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:               supplies
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0030      0.001     -3.090      0.002      -0.005      -0.001
income        -4.713e-07   4.39e-07     -1.075      0.283   -1.33e-06    3.88e-07
education        -0.0109      0.014     -0.761      0.447      -0.039       0.017
homeownership     0.0648      0.042      1.541      0.123      -0.018       0.147
=================================================================================
"""

In [36]:
probit('involved ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.275524
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:               involved   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.08213
Time:                        18:01:33   Log-Likelihood:                -130.05
converged:                       True   LL-Null:                       -141.68
Covariance Type:            nonrobust   LLR p-value:                 0.0001116
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.6086      0.267     -2.276      0.023      -1.133      -0.085
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:               involved
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0028      0.001     -3.606      0.000      -0.004      -0.001
income         1.878e-07   2.87e-07      0.654      0.513   -3.75e-07     7.5e-07
education         0.0132      0.010      1.384      0.166      -0.006       0.032
homeownership    -0.0581      0.029     -1.997      0.046      -0.115      -0.001
=================================================================================
"""

In [37]:
probit('made_plan ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.519771
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:              made_plan   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                0.009292
Time:                        18:01:39   Log-Likelihood:                -245.33
converged:                       True   LL-Null:                       -247.63
Covariance Type:            nonrobust   LLR p-value:                    0.3306
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.4005      0.203     -1.970      0.049      -0.799      -0.002
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:              made_plan
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0018      0.001     -1.813      0.070      -0.004       0.000
income         1.912e-07   4.21e-07      0.455      0.649   -6.33e-07    1.02e-06
education        -0.0119      0.014     -0.817      0.414      -0.040       0.017
homeownership    -0.0015      0.042     -0.036      0.971      -0.084       0.081
=================================================================================
"""

In [38]:
probit('practiced_drills ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.263457
         Iterations 7
                          Probit Regression Results                           
Dep. Variable:       practiced_drills   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                 0.05911
Time:                        18:01:44   Log-Likelihood:                -124.35
converged:                       True   LL-Null:                       -132.16
Covariance Type:            nonrobust   LLR p-value:                  0.003565
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.5977      0.272     -2.194      0.028      -1.132      -0.064
age              -0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:       practiced_drills
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0026      0.001     -3.569      0.000      -0.004      -0.001
income        -2.121e-07   3.02e-07     -0.703      0.482   -8.04e-07    3.79e-07
education         0.0064      0.009      0.687      0.492      -0.012       0.025
homeownership     0.0140      0.028      0.503      0.615      -0.041       0.069
=================================================================================
"""

In [39]:
probit('alerts ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.482972
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                 alerts   No. Observations:                  472
Model:                         Probit   Df Residuals:                      467
Method:                           MLE   Df Model:                            4
Date:                Tue, 08 Apr 2025   Pseudo R-squ.:                0.002387
Time:                        18:01:49   Log-Likelihood:                -227.96
converged:                       True   LL-Null:                       -228.51
Covariance Type:            nonrobust   LLR p-value:                    0.8957
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.0306      0.216     -4.767      0.000      -1.454      -0.607
age               0.

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                 alerts
Method:                          dydx
At:                           overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age               0.0005      0.001      0.586      0.558      -0.001       0.002
income        -2.141e-07   4.06e-07     -0.528      0.598   -1.01e-06    5.81e-07
education         0.0050      0.014      0.367      0.714      -0.022       0.032
homeownership     0.0221      0.040      0.553      0.580      -0.056       0.100
=================================================================================
"""

In [40]:
probit('family_communication ~ age + income + education + homeownership', data = df)

Optimization terminated successfully.
         Current function value: 0.320550
         Iterations 6
                           Probit Regression Results                            
Dep. Variable:     family_communication   No. Observations:                  472
Model:                           Probit   Df Residuals:                      467
Method:                             MLE   Df Model:                            4
Date:                  Tue, 08 Apr 2025   Pseudo R-squ.:                 0.02506
Time:                          18:01:58   Log-Likelihood:                -151.30
converged:                         True   LL-Null:                       -155.19
Covariance Type:              nonrobust   LLR p-value:                    0.1000
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.9338      0.252     -3.711      0.000      -1.427      -0.441
age 

<class 'statsmodels.iolib.summary.Summary'>
"""
        Probit Marginal Effects        
=======================================
Dep. Variable:     family_communication
Method:                            dydx
At:                             overall
=================================================================================
                   dy/dx    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
age              -0.0018      0.001     -2.492      0.013      -0.003      -0.000
income        -9.523e-08   3.19e-07     -0.299      0.765    -7.2e-07    5.29e-07
education         0.0140      0.010      1.331      0.183      -0.007       0.035
homeownership     0.0031      0.031      0.100      0.920      -0.057       0.063
=================================================================================
"""